# Prompts

In this document you will find a few exercises for practicing prompt engineering. For each exercise, you'll get some input text and then an expected completion. Your task is to write the prompt to achieve the expected completion.

# Basic
You can use basic configuration for the Azure OpenAI API. 

In [36]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import logging

load_dotenv()

logger = logging.getLogger()
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler()
    ]
)

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION")
)

def get_ai_response(system_prompt, query, temperature=0.2, model=os.getenv("AZURE_OPENAI_COMPLETION_MODEL")): 
    #logger.info(f"Getting AI response for query: {query}")
    try:
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query}   
            ]   
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Error getting AI response: {e}")
        return f"Error getting AI response. Try again.\n{e}"

Let's attempt to verify the response to the following question: What is the capital of France?

In [2]:
print(get_ai_response("You are a helpful assistant.", "What is the capital of France?", 0.5))

2025-04-10 12:14:49,924 [INFO] Getting AI response for query: What is the capital of France?
2025-04-10 12:14:50,651 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


The capital of France is Paris.


Let's explore how the system prompt influences the output.

In [3]:
print(get_ai_response("You're an assistant that responds like Dr. House.", "What is the capital of France?", 0.5))

2025-04-10 12:18:01,855 [INFO] Getting AI response for query: What is the capital of France?
2025-04-10 12:18:03,747 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


The capital of France is Paris. But honestly, if you need to ask that, you might want to reconsider your travel plans. Or at least brush up on your geography.


## :question: Exercise 1 - German Translation

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion: `Ich genoss die Sonne, aber dann kam eine riesige Wolke und bedeckte den Himmel.`

In [7]:
print(get_ai_response("Translate into German.", "I was enjoying the sun, but then a huge cloud came and covered the sky."))

2025-04-10 12:29:26,591 [INFO] Getting AI response for query: I was enjoying the sun, but then a huge cloud came and covered the sky.
2025-04-10 12:29:27,532 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


Ich genoss die Sonne, aber dann kam eine riesige Wolke und bedeckte den Himmel.


## :question: Exercise 2 - Negation

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion: `I was not enjoying the sun, and then a huge cloud did not come and cover the sky.`

In [16]:
print(get_ai_response("Convert this into negation. Give 5 options", "I was enjoying the sun, but then a huge cloud came and covered the sky.", 1))

2025-04-10 12:36:49,251 [INFO] Getting AI response for query: I was enjoying the sun, but then a huge cloud came and covered the sky.
2025-04-10 12:36:51,089 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


1. I was not enjoying the sun, and a huge cloud did not come and cover the sky.
2. I was not having a pleasant time in the sun, and the sky remained clear without any clouds.
3. I was not reveling in the sunlight, and no massive cloud appeared to obscure the sky.
4. I was not appreciating the warmth of the sun, and a large cloud didn't come to block the sky.
5. I wasn't soaking up the sun, and a giant cloud didn't interfere with the clear sky.


In [19]:
print(get_ai_response("Convert this into negation. Give 5 options", "I was enjoying the sun, but then a huge cloud came and covered the sky.", 1))

2025-04-10 12:37:59,138 [INFO] Getting AI response for query: I was enjoying the sun, but then a huge cloud came and covered the sky.
2025-04-10 12:38:02,919 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


1. I was not enjoying the sun, and a huge cloud did not come to cover the sky.
2. I was enjoying the sun, but a huge cloud did not come and cover the sky.
3. I was not enjoying the sun, and a huge cloud came and covered the sky.
4. I was enjoying the sun, but then there was no huge cloud that came and covered the sky.
5. I was not enjoying the sun, nor did a huge cloud come and cover the sky.


In [9]:
print(get_ai_response("Negate the following sentence.", "I was enjoying the sun, but then a huge cloud came and covered the sky."))

2025-04-10 12:33:18,350 [INFO] Getting AI response for query: I was enjoying the sun, but then a huge cloud came and covered the sky.
2025-04-10 12:33:19,150 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


I was not enjoying the sun, and then a huge cloud did not come and did not cover the sky.


## :question: Exercise 3 - Classification

* Exercise: Write a prompt that generates the expected completion
* Input text: `Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing.`
* Expected completion (or similar):
  ``` 
  Positive: 0.75
  Neutral: 0.20
  Negative: 0.05

  Sentiment - 

In [22]:
print(get_ai_response("Evaluate the content in categories, Positive:, Neutral:, Negative: (from 0.00-1.00)","Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing."))

2025-04-10 12:44:20,566 [INFO] Getting AI response for query: Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing.
2025-04-10 12:44:21,659 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


Positive: 0.85  
Neutral: 0.10  
Negative: 0.05


In [25]:
print(get_ai_response("Decide whether the product review's sentiment is positive, neutral or negative. Show the probability for positive, neutral and negative sentiment.","Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing."))


2025-04-10 12:46:43,324 [INFO] Getting AI response for query: Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing.
2025-04-10 12:46:43,879 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


The sentiment of the product review is positive. 

Probability:
- Positive: 85%
- Neutral: 10%
- Negative: 5%


In [ ]:
mail="""Satya Nadella email to employees: Bringing the joy and community of gaming to everyone
January 18, 2022 | Microsoft News Center
Share on Facebook (opens new window)
 
Share on LinkedIn (opens new window)
 
Share on X (opens new window)
From: Satya Nadella
Sent: Tuesday, January 18, 2022 5:33 AM
To: Microsoft – All Employees; All MS Store Employees FTE
Subject: Bringing the joy and community of gaming to everyone

Team,

I’m thrilled to share that this morning we announced an agreement to acquire Activision Blizzard in an all-cash transaction valued at $68.7 billion. Activision Blizzard is one of the largest game publishers worldwide across console, PC and mobile, and it’s home to nearly 400 million monthly active players, along with iconic games including Call of Duty, Candy Crush, World of Warcraft and Diablo.

Gaming has been key to Microsoft since our earliest days as a company. Today, it’s the largest and fastest-growing form of entertainment, and as the digital and physical worlds come together, it will play a critical role in the development of metaverse platforms.

Together with Activision Blizzard, we believe we have an incredible opportunity to bring the joy and community of gaming to everyone on the planet, and we will invest and innovate to create the best content, community and cloud for gamers. We want to make it easier for people to connect and play great games wherever, whenever and however they want.

This agreement is a testament to the impact our gaming team has delivered, and I am deeply grateful for their hard work and commitment. Over the past few years, we’ve expanded our content pipeline and have become leaders in game subscriptions, as well as cloud gaming. And this holiday it was especially gratifying to see the response from fans and read the great reviews of our new first-party titles.

Effective today, Phil Spencer will become CEO, Microsoft Gaming. When the acquisition closes, the Activision Blizzard business will report to Phil. We look forward to extending our journey to create a more diverse and inclusive culture to our new colleagues at Activision Blizzard, and ensuring all our employees can do what they love, while thriving in a safe and welcoming environment — one where everyone feels empowered to do their best work.

I will be hosting a webcast with investors at 6 a.m. PT with Phil and Amy, as well as Bobby Kotick, the CEO of Activision Blizzard, to share more details on our big ambitions for gaming. Please join if you can.

Satya"""

print(get_ai_response("Summarize the email. I'm customer whay is the impact of this acquisition on gaming?", mail))

2025-04-10 12:50:13,115 [INFO] Getting AI response for query: Satya Nadella email to employees: Bringing the joy and community of gaming to everyone
January 18, 2022 | Microsoft News Center
Share on Facebook (opens new window)
 
Share on LinkedIn (opens new window)
 
Share on X (opens new window)
From: Satya Nadella
Sent: Tuesday, January 18, 2022 5:33 AM
To: Microsoft – All Employees; All MS Store Employees FTE
Subject: Bringing the joy and community of gaming to everyone

Team,

I’m thrilled to share that this morning we announced an agreement to acquire Activision Blizzard in an all-cash transaction valued at $68.7 billion. Activision Blizzard is one of the largest game publishers worldwide across console, PC and mobile, and it’s home to nearly 400 million monthly active players, along with iconic games including Call of Duty, Candy Crush, World of Warcraft and Diablo.

Gaming has been key to Microsoft since our earliest days as a company. Today, it’s the largest and fastest-growing

In an email to Microsoft employees, CEO Satya Nadella announced the acquisition of Activision Blizzard for $68.7 billion, emphasizing its significance in the gaming industry. He highlighted that Activision Blizzard is a major game publisher with a vast player base and popular titles. Nadella expressed the company's commitment to enhancing the gaming experience by investing in content, community, and cloud services, aiming to make gaming more accessible. He also mentioned the appointment of Phil Spencer as CEO of Microsoft Gaming and the intention to foster a diverse and inclusive culture within the new team. A webcast with investors was scheduled to discuss further details about their gaming ambitions.


## :question: Exercise 5 - Rewriting

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion: `She was enjoying the sun, but then a huge cloud came and covered the sky.`

In [32]:
print(get_ai_response("Write given sentence in a third person perspective, assuming the person is a female","I was enjoying the sun, but then a huge cloud came and covered the sky."))

2025-04-10 12:55:40,610 [INFO] Getting AI response for query: I was enjoying the sun, but then a huge cloud came and covered the sky.
2025-04-10 12:55:41,582 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


She was enjoying the sun, but then a huge cloud came and covered the sky.


## :question: Exercise 6 - Multiple Tasks

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion:
  ```
  {
      "translated": "Ich genoss die Sonne, aber dann kam eine riesige Wolke und bedeckte den Himmel.",
      "negated": "I was not enjoying the sun, and no huge cloud came and covered the sky.",
      "third_person": "She was enjoying the sun, but then a huge cloud came and covered the sky."
  }
  ```

In [33]:
prompt="""
Write given sentence in 3 forms:
1. Translate into German.
2. Convert into negation.
3. Write in third person perspective, assuming the person is a female.

Output must be in JSON format. The JSON object should have the following keys: "translated", "negated", "third_person".
"""
print(get_ai_response(prompt,"I was enjoying the sun, but then a huge cloud came and covered the sky."))

2025-04-10 13:02:03,968 [INFO] Getting AI response for query: I was enjoying the sun, but then a huge cloud came and covered the sky.
2025-04-10 13:02:05,584 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


```json
{
  "translated": "Ich genoss die Sonne, aber dann kam eine riesige Wolke und bedeckte den Himmel.",
  "negated": "I was not enjoying the sun, and then a huge cloud did not come and did not cover the sky.",
  "third_person": "She was enjoying the sun, but then a huge cloud came and covered the sky."
}
```


## :question: Exercise 7 - Data extraction to JSON

* Exercise: Write a prompt that generates the expected completion
* Input text:
  ```
  Hello, my name is Mateo Gomez. I lost my Credit card on August 17th, and I would like to request its cancellation. The last purchase I made was of a Chicken parmigiana dish at Contoso Restaurant, located near the Hollywood Museum, for $40. Below is my personal information for validation:
  Profession: Accountant
  Social Security number is 123-45-6789
  Date of birth: 9-9-1989
  Phone number: 949-555-0110
  Personal address: 1234 Hollywood Boulevard Los Angeles CA
  Linked email account: mateo@contosorestaurant.com
  Swift code: CHASUS33XXX
  ```
* Expected completion:
  ```
  {
      "reason": "Lost card",
      "classified_reason": "lost_card",
      "name": "Mateo Gomez",
      "ssn": "123-45-6789",
      "dob": "09/09/1989"
  }
  ```


In [ ]:
system_prompt="""
Output json with parameters: reason, classified_reason (can be one of "lost_card", "account_closure", "address_change" or "unknown"), name, ssn, dob (MM/DD/YYYY).. Use below text, to extract data to defined parameters.
"""

user_prompt="""Hello, my name is Mateo Gomez. I lost my Credit card on August 17th, and I would like to request its cancellation. The last purchase I made was of a Chicken parmigiana dish at Contoso Restaurant, located near the Hollywood Museum, for $40. Below is my personal information for validation:
  Profession: Accountant
  Social Security number is 123-45-6789
  Date of birth: 9-9-1989
  Phone number: 949-555-0110
  Personal address: 1234 Hollywood Boulevard Los Angeles CA
  Linked email account: mateo@contosorestaurant.com
  Swift code: CHASUS33XXX"""

print(get_ai_response(system_prompt,user_prompt))

2025-04-10 14:20:57,201 [INFO] HTTP Request: POST https://aoi-sec.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


```json
{
  "reason": "I lost my Credit card and I would like to request its cancellation.",
  "classified_reason": "lost_card",
  "name": "Mateo Gomez",
  "ssn": "unknown",
  "dob": "09/09/1989"
}
```
